# References

* [Mask RCNN Benchmark: model definitions and skeleton networks](http://echo.etri.re.kr:8090/display/~kimkk/Mask+RCNN+Benchmark%3A+model+definitions+and+skeleton+networks) - code by code analysis

----

# import caffe layers and params 

In [3]:
import caffe
from caffe import layers as L, params as P

----

### helper to build a conv -> BN -> relu block

mask rcnn에서 사용하는 FrozenBatchNorm2d 방식으로 구현할 것.
* [Why FrozenBatchNorm2d in ResNet?](http://echo.etri.re.kr:8090/pages/viewpage.action?pageId=78088692)
* https://github.com/facebookresearch/maskrcnn-benchmark/issues/267
* [Batch Normalization 설명 및 구현](http://echo.etri.re.kr:8090/pages/viewpage.action?pageId=78088844)

Caffe BatchNorm vs Torch BatchNorm2d  
* [During test phase, the output of Batch Normalization layer is not equal to (input-mean)/sqrt(var + eps)?](https://github.com/BVLC/caffe/issues/4885)
> `net.params['bn_1'][0].data` : mean  
> `net.params['bn_1'][1].data` : variance   
> `net.params['bn_1'][2].data` : scores a scale factor, which I should divide from each mean and variance at this layer.    

So in Caffe, the output of BN layer calculated by: 

$
\begin{aligned}
\frac{\frac{ \text{input} - \text{mean} }{ \text{scale_factor} }}{\sqrt { \frac{ \text{var }}{ \text{scale_factor } } + \epsilon}}
\end{aligned}
$

batchnorm 없이 scale 레이어로만해도  될듯...

**Batch Norm Layer:**  
* gamma --> Var (1)   
* beta  --> mean (0)   
* mean  --> moving average fraction (=1)   

**Scale Layer:**
* scale --> 논문에서의 gamma   
* bias  --> 논문에서의 beta   


```python
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.
import torch
from torch import nn
 
 
class FrozenBatchNorm2d(nn.Module):
    """
    BatchNorm2d where the batch statistics and the affine parameters
    are fixed
    """
 
    def __init__(self, n):
        super(FrozenBatchNorm2d, self).__init__()
        self.register_buffer("weight", torch.ones(n))
        self.register_buffer("bias", torch.zeros(n))
        self.register_buffer("running_mean", torch.zeros(n))
        self.register_buffer("running_var", torch.ones(n))
 
    def forward(self, x):
        # Cast all fixed parameters to half() if necessary
        if x.dtype == torch.float16:
            self.weight = self.weight.half()
            self.bias = self.bias.half()
            self.running_mean = self.running_mean.half()
            self.running_var = self.running_var.half()
 
        scale = self.weight * self.running_var.rsqrt()
        bias = self.bias - self.running_mean * scale
        scale = scale.reshape(1, -1, 1, 1)
        bias = bias.reshape(1, -1, 1, 1)
        return x * scale + bias
```


In [4]:

def conv_bn_scale(bottom, nout, bias_term=False, **kwargs):
    '''Helper to build a conv -> BN -> relu block.
    '''
    conv = L.Convolution(bottom, num_output=nout, bias_term=bias_term,
                         **kwargs)
    
    # https://github.com/facebookresearch/maskrcnn-benchmark/blob/master/maskrcnn_benchmark/layers/batch_norm.py
    bn = L.BatchNorm(conv, use_global_stats=True, in_place=True)
    scale = L.Scale(bn, bias_term=True, in_place=True)
    return conv, bn, scale




----

### helper to build basic resnet block 

Original Code Source
* [https://gist.github.com/kyamagu/80c343a14ae4564ef79b53f0b01cd57e](https://gist.github.com/kyamagu/80c343a14ae4564ef79b53f0b01cd57e)
* [https://gist.github.com/rezoo/a1c8d1459b222fc5658f](https://gist.github.com/rezoo/a1c8d1459b222fc5658f)


----

### why downsample?

When the input and output channels of the bottleneck are not equal, a certain strategy needs to be adopted.

In the original paper, there are three strategies A, B, and C. Here we use strategy B (also recommended by the original paper)

That is, **projection shortcuts** are used only when the number of input and output channels are not equal, 

- the parameter matrix mapping is used to make the input and output channels equal

----

### helper to build resnet50 

* [resnet50 overall](https://i.stack.imgur.com/gI4zT.png)
* [resnet50 layers in table](https://i.stack.imgur.com/FoH7Z.jpg)
* [resnet50 structure](https://www.researchgate.net/publication/331364877/figure/fig3/AS:741856270901252@1553883726825/Left-ResNet50-architecture-Blocks-with-dotted-line-represents-modules-that-might-be.png)
* [resnet50_structure1](https://www.mdpi.com/plants/plants-10-00031/article_deploy/html/images/plants-10-00031-g010.png)
* [FrozenBatchNorm2d 소스 코드](https://github.com/facebookresearch/maskrcnn-benchmark/blob/master/maskrcnn_benchmark/layers/batch_norm.py)

### lomin detection model structure


**Note**  
> This is just structure of detection model v2, its structure does not imply any order of layers.  
> In this structure analysis, some layers those not include learnable parameters, such as ReLU or maxpool are hidden.
* source : [detection model v2 structure](http://echo.etri.re.kr:8090/display/~kimkk/detection+model+v2+structure)

```java
(lomin) kimkk@alpha ~/lomin $ vi model_structure.txt
 
GenerallizedRCNN  //model
  │
  ├─── backbone // model.backbone
  │     │
  │     ├─── body // model.backbone.body
  │     │     ├── stem // model.backbone.body.stem
  │     │     │    │
  │     │     │    ├── conv1  // model.backbone.body.stem.conv1, def: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  │     │     │    └── bn1    // model.backbone.body.stem.conv1, def: FrozenBatchNorm2d()
  │     │     │
  │     │     ├── layer1 //model.backbone.body.layer1
  │     │     │    │
  │     │     │    ├─── 0     // model.backbone.body.layer1.0
  │     │     │    │    ├── downsample // model.backbone.body.layer1.0.downsample
  │     │     │    │    │    │
  │     │     │    │    │    ├── 0     // model.backbone.body.layer1.0.downsample.0,  def: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False) x
  │     │     │    │    │    └── 1     // model.backbone.body.layer1.0.downsample.1,  def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv1      // model.backbone.body.layer1.0.conv1,         def: Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1        // model.backbone.body.layer1.0.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2      // model.backbone.body.layer1.0.conv2,         def: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2        // model.backbone.body.layer1.0.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3      // model.backbone.body.layer1.0.conv3,         def: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3        // model.backbone.body.layer1.0.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    ├─── 1 // model.backbone.body.layer1.1
  │     │     │    │    ├── conv1      // model.backbone.body.layer1.1.conv1,         def: Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1        // model.backbone.body.layer1.1.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2      // model.backbone.body.layer1.1.conv2,         def: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2        // model.backbone.body.layer1.1.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3      // model.backbone.body.layer1.1.conv3,         def: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3        // model.backbone.body.layer1.1.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    └─── 2 // model.backbone.body.layer1.2
  │     │     │         ├── conv1       // model.backbone.body.layer1.2.conv1,         def: Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │         ├── bn1         // model.backbone.body.layer1.2.bn1,           def: FrozenBatchNorm2d()
  │     │     │         ├── conv2       // model.backbone.body.layer1.2.conv2,         def: Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │         ├── bn2         // model.backbone.body.layer1.2.bn2,           def: FrozenBatchNorm2d()
  │     │     │         ├── conv3       // model.backbone.body.layer1.2.conv3,         def: Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │         └── bn3         // model.backbone.body.layer1.2.conv2,         def: FrozenBatchNorm2d()
  │     │     │
  │     │     ├── layer2 // model.backbone.body.layer2
  │     │     │    │
  │     │     │    ├─── 0 // model.backbone.body.layer2.0)
  │     │     │    │    ├── downsample  // model.backbone.body.layer2.0.downsample)
  │     │     │    │    │    │
  │     │     │    │    │    ├── 0      // model.backbone.body.layer2.0.downsample.0 , def: Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
  │     │     │    │    │    └── 1      // model.backbone.body.layer2.0.downsample.1,  def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv1       // model.backbone.body.layer2.0.conv1,         def: Conv2d(256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
  │     │     │    │    ├── bn1         // model.backbone.body.layer2.0.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2       // model.backbone.body.layer2.0.conv2,         def: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2         // model.backbone.body.layer2.0.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3       // model.backbone.body.layer2.0.conv3,         def: Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3         // model.backbone.body.layer2.0.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    ├─── 1 // model.backbone.body.layer2.1
  │     │     │    │    ├── conv1       // model.backbone.body.layer2.1.conv1,         def: Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1         // model.backbone.body.layer2.1.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2       // model.backbone.body.layer2.1.conv2,         def: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2         // model.backbone.body.layer2.1.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3       // model.backbone.body.layer2.1.conv3,         def: Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3         // model.backbone.body.layer2.1.bn3,           def:# FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    ├─── 2 // model.backbone.body.layer2.2
  │     │     │    │    ├── conv1       // model.backbone.body.layer2.2.conv1,         def: Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1         // model.backbone.body.layer2.2.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2       // model.backbone.body.layer2.2.conv2,         def: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2         // model.backbone.body.layer2.2.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3       // model.backbone.body.layer2.2.conv3,         def: Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3         // model.backbone.body.layer2.2.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    └─── 3 // model.backbone.body.layer2.3
  │     │     │         ├── conv1       // model.backbone.body.layer2.3.conv1,         def: Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │         ├── bn1         // model.backbone.body.layer2.3.bn1,           def: FrozenBatchNorm2d()
  │     │     │         ├── conv2       // model.backbone.body.layer2.3.conv2,         def: Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │         ├── bn2         // model.backbone.body.layer2.3.bn2,           def: FrozenBatchNorm2d()
  │     │     │         ├── conv3       // model.backbone.body.layer2.3.conv3,         def: # Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │         └── bn3         // model.backbone.body.layer2.3.bn3,           def: # FrozenBatchNorm2d()
  │     │     │
  │     │     ├── layer3 // model.backbone.body.layer3
  │     │     │    │
  │     │     │    ├─── 0 // model.backbone.body.layer3.0
  │     │     │    │    ├── downsample // model.backbone.body.layer3.0.downsamples
  │     │     │    │    │    │
  │     │     │    │    │    ├── 0     // model.backbone.body.layer3.0.downsamples.0, def: Conv2d(512, 1024, kernel_size=(1, 1), stride=(2, 2), bias=False)
  │     │     │    │    │    └── 1     // model.backbone.body.layer3.0.downsamples.0, def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv1      // model.backbone.body.layer3.0.conv1,         def: Conv2d(512, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
  │     │     │    │    ├── bn1        // model.backbone.body.layer3.0.bn1,           def: # FrozenBatchNorm2d()
  │     │     │    │    ├── conv2      // model.backbone.body.layer3.0.conv2,         def: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2        // model.backbone.body.layer3.0.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3      // model.backbone.body.layer3.0.conv3,         def: Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3        // model.backbone.body.layer3.0.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    ├─── 1 // model.backbone.body.layer3.1
  │     │     │    │    ├── conv1      // model.backbone.body.layer3.1.conv1,         def: Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1        // model.backbone.body.layer3.1.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2      // model.backbone.body.layer3.1.conv2,         def: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2        // model.backbone.body.layer3.1.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3      // model.backbone.body.layer3.1.conv3,         def: Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3        // model.backbone.body.layer3.1.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    ├─── 2 // model.backbone.body.layer3.2
  │     │     │    │    ├── conv1      // model.backbone.body.layer3.2.conv1,         def: Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1        // model.backbone.body.layer3.2.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2      // model.backbone.body.layer3.2.conv2,         def: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2        // model.backbone.body.layer3.2.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3      // model.backbone.body.layer3.2.conv3,         def: Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3        // model.backbone.body.layer3.2.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    ├─── 3 // model.backbone.body.layer3.3
  │     │     │    │    ├── conv1      // model.backbone.body.layer3.3.conv1,         def: Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1        // model.backbone.body.layer3.3.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2      // model.backbone.body.layer3.3.conv2,         def: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2        // model.backbone.body.layer3.3.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3      // model.backbone.body.layer3.3.conv3,         def: Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3        // model.backbone.body.layer3.3.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    ├─── 4 (model.backbone.body.layer3.4)
  │     │     │    │    ├── conv1      // model.backbone.body.layer3.4.conv1,         def: Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    ├── bn1        // model.backbone.body.layer3.4.bn1,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv2      // model.backbone.body.layer3.4.conv2,         def: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │    │    ├── bn2        // model.backbone.body.layer3.4.bn2,           def: FrozenBatchNorm2d()
  │     │     │    │    ├── conv3      // model.backbone.body.layer3.4.conv3,         def: Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │    │    └── bn3        // model.backbone.body.layer3.4.bn3,           def: FrozenBatchNorm2d()
  │     │     │    │
  │     │     │    └─── 5 // model.backbone.body.layer3.5
  │     │     │         ├── conv1       // model.backbone.body.layer3.5.conv1,         def: Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │         ├── bn1         // model.backbone.body.layer3.5.bn1,           def: FrozenBatchNorm2d()
  │     │     │         ├── conv2       // model.backbone.body.layer3.5.conv2,         def: Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │     │         ├── bn2         // model.backbone.body.layer3.5.bn2,           def: FrozenBatchNorm2d()
  │     │     │         ├── conv3       // model.backbone.body.layer3.5.conv3,         def: Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │     │         └── bn3         // model.backbone.body.layer3.5.bn3,           def: FrozenBatchNorm2d()
  │     │     │
  │     │     └── layer4 // model.backbone.body.layer4
  │     │          │
  │     │          ├─── 0 // model.backbone.body.layer4.0
  │     │          │    ├── downsample  // model.backbone.body.layer4.0.downsample
  │     │          │    │   │
  │     │          │    │   ├── 0       // model.backbone.body.layer4.0.downsample.0,  def: conv1 Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
  │     │          │    │   └── 1       // model.backbone.body.layer4.0.downsample.1,  def: FrozenBatchNorm2d()
  │     │          │    ├── conv1       // model.backbone.body.layer4.0.conv1,         def: Conv2d(1024, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
  │     │          │    ├── bn1         // model.backbone.body.layer4.0.bn1,           def: FrozenBatchNorm2d()
  │     │          │    ├── conv2       // model.backbone.body.layer4.0.conv2,         def: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │          │    ├── bn2         // model.backbone.body.layer4.0.bn2,           def: FrozenBatchNorm2d()
  │     │          │    ├── conv3       // model.backbone.body.layer4.0.conv3,         def: Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │          │    └── bn3         // model.backbone.body.layer4.0.bn3,           def: FrozenBatchNorm2d()
  │     │          │
  │     │          ├─── 1 // model.backbone.body.layer4.1
  │     │          │    ├── conv1       // model.backbone.body.layer4.1.conv1,         def: Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │          │    ├── bn1         // model.backbone.body.layer4.1.bn1,           def: FrozenBatchNorm2d()
  │     │          │    ├── conv2       // model.backbone.body.layer4.1.conv2,         def: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │          │    ├── bn2         // model.backbone.body.layer4.1.bn2,           def: FrozenBatchNorm2d()
  │     │          │    ├── conv3       // model.backbone.body.layer4.1.conv3,         def: Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │          │    └── bn3         // model.backbone.body.layer4.1.bn3,           def: FrozenBatchNorm2d()
  │     │          │
  │     │          └─── 2 // model.backbone.body.layer4.2
  │     │               ├── conv1       // model.backbone.body.layer4.2.conv1,         def: Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │               ├── bn1         // model.backbone.body.layer4.2.bn1,           def: FrozenBatchNorm2d()
  │     │               ├── conv2       // model.backbone.body.layer4.2.conv2,         def: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  │     │               ├── bn2         // model.backbone.body.layer4.2.bn2,           def: FrozenBatchNorm2d()
  │     │               ├── conv3       // model.backbone.body.layer4.2.conv3,         def: Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  │     │               └── bn3         // model.backbone.body.layer4.2.bn3,           def: FrozenBatchNorm2d()
  │     │
  │     └─── fpn // model.backbone.fpn
  │           ├── fpn_inner2            // model.backbone.fpn.fpn_inner2,              def: Conv2d(512, 1024, kernel_size=(1, 1), stride=(1, 1))
  │           ├── fpn_layer2            // model.backbone.fpn.fpn_layer2,              def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  │           ├── fpn_inner3            // model.backbone.fpn.fpn_inner3,              def: Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
  │           ├── fpn_layer3            // model.backbone.fpn.fpn_layer3,              def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  │           ├── fpn_inner4            // model.backbone.fpn.fpn_inner4,              def: Conv2d(2048, 1024, kernel_size=(1, 1), stride=(1, 1))
  │           ├── fpn_layer4            // model.backbone.fpn.fpn_layer4,              def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  │           └── top_blocks // model.backbone.fpn.topblocks
  │                 │
  │                 ├─── p6             // model.backbone.fpn.topblocks.p6,            def: Conv2d(2048, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  │                 └─── p7             // model.backbone.fpn.topblocks.p7,            def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  │
  └─── rpn //  model.rpn
        │
        ├─── anchor_generator // model.rpn.anchor_generator
        │           └─── cell_anchors  // model.rpn.anchor_generator.cell_achors,      def: BufferList()
        │
        ├─── head // model.rpn.head
        │           ├─── cls_tower    // model.rpn.head.cls_tower
        │           │     ├─── 0      // model.rpn.head.cls_tower.0,                   def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │           │     ├─── 1      // model.rpn.head.cls_tower.1,                   def: ReLU()
        │           │     ├─── 2      // model.rpn.head.cls_tower.2,                   def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │           │     ├─── 3      // model.rpn.head.cls_tower.3,                   def: ReLU()
        │           │     ├─── 4      // model.rpn.head.cls_tower.4,                   def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU()
        │           │     ├─── 5      // model.rpn.head.cls_tower.5,                   def: ReLU()
        │           │     ├─── 6      // model.rpn.head.cls_tower.6,                   def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │           │     └─── 7      // model.rpn.head.cls_tower.7,                   def: ReLU()
        │           │
        │           ├─── bbox_tower   // model.rpn.head.bbox_tower
        │           │     ├─── 0      // model.rpn.head.bbox_tower.0,                  def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │           │     ├─── 1      // model.rpn.head.bbox_tower.1,                  def: ReLU()
        │           │     ├─── 2      // model.rpn.head.bbox_tower.2,                  def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │           │     ├─── 3      // model.rpn.head.bbox_tower.3,                  def: ReLU()
        │           │     ├─── 4      // model.rpn.head.bbox_tower.4,                  def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) ReLU()
        │           │     ├─── 5      // model.rpn.head.bbox_tower.5,                  def: ReLU()
        │           │     ├─── 6      // model.rpn.head.bbox_tower.6,                  def: Conv2d(1024, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │           │     └─── 7      // model.rpn.head.bbox_tower.7,                  def: ReLU()
        │           │
        │           ├─── cls_logits   // model.rpn.head.cls_logits,                    def: Conv2d(1024, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │           └─── bbox_pred    // model.rpn.head.bbox_pre,                      def: Conv2d(1024, 36, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        │
        └─── box_selector_test        // model.rpn.box_selector_test,                  def: RetinaNetPostProcessor()
```

In [18]:
def resnet50(n, bottom):
    '''ResNet 50 layers.
    '''
    
    stage_prefix = "backbone_body"
    
    #-------------------------------------
    # model.backbone.body.stem (stage 0)
    #-------------------------------------
    stage_name = "stem"
    stage = f'{stage_prefix}_{stage_name}_'
    
    n[stage + 'conv1'], n[stage + 'bn1'], n[stage + 'scale'] =_conv_bn_scale( bottom, 64, bias_term=False, kernel_size=7, pad=3, stride=2)
    n[stage + 'relu'] = L.ReLU(n[stage + 'scale'])
    n[stage + 'maxpool'] = L.Pooling( n[stage + 'relu'], kernel_size=3, stride=2, pool=P.Pooling.MAX)
    
    #-------------------------------------
    # model.backbone.layer1 (stage 1)
    #-------------------------------------
    pre_stage = stage
    stage_name = "layer1"
    stage = f'{stage_prefix}_{stage_name}'
        
    resnet_block(stage, '0', n, n[pre_stage + 'maxpool'], 64, downsample_branch=True, initial_stride=1)
    resnet_block(stage, '1', n, n[stage + '_0_downsample_0_relu'], 64)
    resnet_block(stage, '2', n, n[stage + '_1_relu'], 64)
    
    #-------------------------------------
    # model.backbone.layer2 (stage 2)
    #-------------------------------------
    pre_stage = stage
    stage_name = "layer2"
    stage = f'{stage_prefix}_{stage_name}'
    
    resnet_block(state, '0', n, n[pre_stage + 'relu'], 128, downsample_branch=True)
    resnet_block(stage, '1', n, n[stage + '_0_downsample_0_relu'], 128)
    resnet_block(stage, '2', n, n[stage + '_1_relu'], 128)
    resnet_block(stage, '3', n, n[stage + '_2_relu'], 128)
    
    #_resnet_block(prefix, '0', n, n.res2c_relu, 128, branch1=True)
    #_resnet_block(prefix, '1', n, n.res3a_relu, 128)
    #_resnet_block(prefix, '2', n, n.res3b_relu, 128)
    #_resnet_block(prefix, '3', n, n.res3c_relu, 128)

     #-------------------------------------
    # model.backbone.layer3 (stage 3)
    #-------------------------------------
    pre_stage = stage
    stage_name = "layer3"
    stage = f'{stage_prefix}_{stage_name}'
    
    resnet_block(prefix, '0', n, n[pre_stage + 'relu'], 256, downsample_branch=True)
    resnet_block(prefix, '1', n, n[stage + '0_downsample_0_relu'], 256)
    resnet_block(prefix, '2', n, n[stage + '_1_relu'], 256)
    resnet_block(prefix, '3', n, n[stage + '_2_relu'], 256)
    resnet_block(prefix, '4', n, n[stage + '_3_relu'], 256)
    resnet_block(prefix, '5', n, n[stage + '_4_relu'], 256)

    #-------------------------------------
    # model.backbone.layer4 (stage 4)
    #-------------------------------------
    pre_stage = stage
    stage_name = "layer4"
    stage = f'{stage_prefix}_{stage_name}'
    
    stage= f"backbone_body_layer4"  
    resnet_block(prefix, '0', n, n[pre_stage + 'relu'], 512, downsample_branch=True)
    resnet_block(prefix, '1', n, n[stage + '_0_downsample_0_relu'], 512)
    resnet_block(prefix, '2', n, n[stage + '_1_relu'], 512)

    #-------------------------------------
    # Polling after stage 4
    #-------------------------------------
    pre_stage = stage
    stage_name = "pool"
    stage = f'{stage_prefix}_{stage_name}'    
    
    n[stage] = L.Pooling(n[pre_stage + 'relu'], kernel_size=7, stride=1, pool=P.Pooling.AVE)

    return n.to_proto()



In [19]:
n = caffe.NetSpec()
n.data = L.DummyData(shape=[dict(dim=[1, 3, 224, 224])])

res50_proto = resnet50(n, n.data)

#print(str(res50_proto))

NameError: name 'relu22' is not defined